Anvil code which is connected with YOLOv8 training model

In [ ]:
from ._anvil_designer import MainFormTemplate
from anvil import *
import plotly.graph_objects as go
import anvil.server
import datetime

class MainForm(MainFormTemplate):
    def __init__(self, **properties):
        self.init_components(**properties)
        
        # Initially hide all results, decision buttons, and the prompt label
        self.label_dominant_color.visible = False
        self.label_dish_suggestion.visible = False
        self.label_olive_message.visible = False
        self.button_yes.visible = False
        self.button_no.visible = False
        self.label_prompt.visible = False
        self.button_generate_report.visible = False  # Add this line
        
        # Link button events to their handlers
        self.button_yes.set_event_handler('click', self.button_yes_click)
        self.button_no.set_event_handler('click', self.button_no_click)
        self.button_generate_report.set_event_handler('click', self.generate_report)  # Add this line

    def file_loader_1_change(self, **event_args):
        uploaded_file = self.file_loader_1.file
        if uploaded_file:
            self.image_display.source = uploaded_file
            # Show the prompt and buttons to ask user for confirmation
            self.label_prompt.text = "Do you want to analyze the image and see the results?"
            self.label_prompt.visible = True
            self.button_yes.visible = True
            self.button_no.visible = True
            # Hide results until user confirms
            self.label_dominant_color.visible = False
            self.label_dish_suggestion.visible = False
            self.label_olive_message.visible = False
            self.button_generate_report.visible = False  # Add this line

    def button_yes_click(self, **event_args):
        uploaded_file = self.file_loader_1.file
        if uploaded_file:
            # Fetch and display results
            self.process_and_display_results(uploaded_file)

    def button_no_click(self, **event_args):
        self.file_loader_1.clear()
        self.reset_display()

    def process_and_display_results(self, uploaded_file):
        try:
            # Call the server-side function 'classify_image' and get the results
            result = anvil.server.call('classify_image', uploaded_file)
            olive_color = result.get('olive_color', 'N/A')
            olive_color_name = result.get('olive_color_name', 'N/A')
            olive_color_message = result.get('olive_color_message', '')
            preservation_tips = result.get('preservation_tips', 'N/A')
            health_tips = result.get('health_tips', 'N/A')
            dish_suggestion = result.get('dish_suggestion', 'N/A')
            
            # Display the results
            self.label_dominant_color.text = f"Olive Color: {olive_color} ({olive_color_name})"
            self.label_dish_suggestion.text = f"Suggested Dish: {dish_suggestion}"
            self.label_olive_message.text = f"{olive_color_message}\n\nPreservation Tips: {preservation_tips}\n\nHealth Tips: {health_tips}"
            
            # Make sure the result labels are visible
            self.label_dominant_color.visible = True
            self.label_dish_suggestion.visible = True
            self.label_olive_message.visible = True

            # Show alert notification based on ripeness
            ripeness_message = self.get_ripeness_notification(olive_color_message)
            if ripeness_message:
                self.show_alert_notification(ripeness_message)

            # Show generate report button
            self.button_generate_report.visible = True  # Add this line

            # Save the result for generating report
            self.result_for_report = result  # Add this line

        except Exception as e:
            # Display error message if something goes wrong
            self.label_olive_message.text = f"Error processing the image: {str(e)}"
            self.label_olive_message.visible = True

        # Hide the confirmation buttons after processing
        self.button_yes.visible = False
        self.button_no.visible = False
        self.label_prompt.visible = False

    def get_ripeness_notification(self, olive_color_message):
        """
        Determine the appropriate message for the alert notification based on olive ripeness.
        """
        if 'unripe' in olive_color_message.lower():
            return "The olives are unripe. Preferably, they should be classified based on their ripeness for olive oil."
        elif 'partially ripe' in olive_color_message.lower():
            return "The olives are partially ripe. Preferably, they should be classified based on their ripeness for olive oil."
        elif 'fully ripe' in olive_color_message.lower():
            return "The olives are fully ripe. Preferably, they should be classified based on their ripeness for olive oil."
        elif 'overripe' in olive_color_message.lower():
            return "The olives are overripe. Preferably, they should be classified based on their ripeness for olive oil."
        else:
            return None

    def show_alert_notification(self, message):
        """
        Show an alert notification with the given message.
        """
        alert(message)

    def generate_report(self, **event_args):
        """
        Generate a PDF report based on the analysis results
        """
        if hasattr(self, 'result_for_report'):
            result = self.result_for_report
            
            # Get current date and time
            current_date = datetime.datetime.now().strftime('%A, %B %d, %Y')
            current_time = datetime.datetime.now().strftime('%H:%M:%S')
            
            pdf_report = anvil.server.call(
                'generate_report',
                result['olive_color'],
                result['olive_color_name'],
                result['olive_color_message'],
                result['preservation_tips'],
                result['health_tips'],
                result['dish_suggestion'],
                current_date,
                current_time
            )
            download(pdf_report)

    def reset_display(self):
        # Reset or hide UI components as needed
        self.button_yes.visible = False
        self.button_no.visible = False
        self.label_dominant_color.visible = False
        self.label_dish_suggestion.visible = False
        self.label_olive_message.visible = False
        self.label_prompt.visible = False
        self.button_generate_report.visible = False  # Add this line